In [17]:
import warnings
warnings.filterwarnings('ignore')
import importlib

In [71]:
import pandas as pd
from pandas import CategoricalDtype
from sklearn.model_selection import train_test_split, KFold
from sklearn.model_selection import cross_val_score
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from lightgbm import LGBMClassifier

import Features
import Models
import Eval

In [82]:
importlib.reload(Features)
importlib.reload(Models)
importlib.reload(Eval)

<module 'Eval' from '/Users/zoos/PycharmProjects/Machine predictive Maintenance/Eval.py'>

### Summary, Best model is lightGBM with balanced accuracy of 
classfier average accuracy for failure or not:  0.8856529855272048

classfier average accuracy for failure type:  0.6291928774454293

### Data

In [73]:
df = pd.read_csv('predictive_maintenance.csv')

In [74]:
df.head()

,UDI,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Target,Failure Type
0,1,M14860,M,298.1,308.6,1551,42.8,0,0,No Failure
1,2,L47181,L,298.2,308.7,1408,46.3,3,0,No Failure
2,3,L47182,L,298.1,308.5,1498,49.4,5,0,No Failure
3,4,L47183,L,298.2,308.6,1433,39.5,7,0,No Failure
4,5,L47184,L,298.2,308.7,1408,40.0,9,0,No Failure


In [75]:
df.columns = [
    "UDI",
    "Product ID",
    "Type",
    "Air_temperature_K",
    "Process_temperature_K",
    "Rotational_speed_rpm",
    "Torque_Nm",
    "Tool_wear_min",
    "Target",
    "Failure_Type"
]

In [76]:
df.shape

(10000, 10)

In [77]:
df.columns

Index(['UDI', 'Product ID', 'Type', 'Air_temperature_K',
       'Process_temperature_K', 'Rotational_speed_rpm', 'Torque_Nm',
       'Tool_wear_min', 'Target', 'Failure_Type'],
      dtype='object')

In [78]:
df.Target.unique()

array([0, 1])

In [80]:
df['Failure_Type'].unique()

array(['No Failure', 'Power Failure', 'Tool Wear Failure',
       'Overstrain Failure', 'Random Failures',
       'Heat Dissipation Failure'], dtype=object)

In [81]:
train, test = train_test_split(df)

### Cross-validation, k-fold and baseline

In [43]:
errs1, errs2 = Eval.evaluate(train)
print("Dummy classfier average accuracy for failure or not: ", np.mean(errs1))
print("Dummy classfier average accuracy for failure type: ", np.mean(errs2))

Fold 0:
  Train: index=[1500 1501 1502 ... 7497 7498 7499]
  Test:  index=[   0    1    2 ... 1497 1498 1499]
  Train err1: 0.5
  Test err1: 0.5
  Train err2: 0.16666666666666666
  Test err2: 0.16666666666666666
Fold 1:
  Train: index=[   0    1    2 ... 7497 7498 7499]
  Test:  index=[1500 1501 1502 ... 2997 2998 2999]
  Train err1: 0.5
  Test err1: 0.5
  Train err2: 0.16666666666666666
  Test err2: 0.16666666666666666
Fold 2:
  Train: index=[   0    1    2 ... 7497 7498 7499]
  Test:  index=[3000 3001 3002 ... 4497 4498 4499]
  Train err1: 0.5
  Test err1: 0.5
  Train err2: 0.16666666666666666
  Test err2: 0.2
Fold 3:
  Train: index=[   0    1    2 ... 7497 7498 7499]
  Test:  index=[4500 4501 4502 ... 5997 5998 5999]
  Train err1: 0.5
  Test err1: 0.5
  Train err2: 0.16666666666666666
  Test err2: 0.16666666666666666
Fold 4:
  Train: index=[   0    1    2 ... 5997 5998 5999]
  Test:  index=[6000 6001 6002 ... 7497 7498 7499]
  Train err1: 0.5
  Test err1: 0.5
  Train err2: 0.1666666

### LogisticRegression

In [67]:
# Without maths features
errs1, errs2 = Eval.evaluate(train, model1=LogisticRegression(), model2=LogisticRegression())
print(" classfier average accuracy for failure or not: ", np.mean(errs1))
print(" classfier average accuracy for failure type: ", np.mean(errs2))

Fold 0:
  Train: index=[1500 1501 1502 ... 7497 7498 7499]
  Test:  index=[   0    1    2 ... 1497 1498 1499]
  Train err1: 0.5814419955044955
  Test err1: 0.5925263764570272
  Train err2: 0.22798080095736004
  Test err2: 0.19748579043267792
Fold 1:
  Train: index=[   0    1    2 ... 7497 7498 7499]
  Test:  index=[1500 1501 1502 ... 2997 2998 2999]
  Train err1: 0.6109044434486821
  Test err1: 0.5652358691032724
  Train err2: 0.2305476942843253
  Test err2: 0.22905156537753224
Fold 2:
  Train: index=[   0    1    2 ... 7497 7498 7499]
  Test:  index=[3000 3001 3002 ... 4497 4498 4499]
  Train err1: 0.5805462451308996
  Test err1: 0.5996563573883161
  Train err2: 0.30478233000485005
  Test err2: 0.2674886305321088
Fold 3:
  Train: index=[   0    1    2 ... 7497 7498 7499]
  Test:  index=[4500 4501 4502 ... 5997 5998 5999]
  Train err1: 0.5880740712912271
  Test err1: 0.6046972795439053
  Train err2: 0.27569393642162676
  Test err2: 0.3321948998178506
Fold 4:
  Train: index=[   0    1  

In [66]:
# With maths
errs1, errs2 = Eval.evaluate(train, features=Features.make_features, model1=LogisticRegression(), model2=LogisticRegression())
print(" classfier average accuracy for failure or not: ", np.mean(errs1))
print(" classfier average accuracy for failure type: ", np.mean(errs2))

Fold 0:
  Train: index=[1500 1501 1502 ... 7497 7498 7499]
  Test:  index=[   0    1    2 ... 1497 1498 1499]
  Train err1: 0.6147602397602397
  Test err1: 0.6345522765499476
  Train err2: 0.23231940061339082
  Test err2: 0.18779744907671805
Fold 1:
  Train: index=[   0    1    2 ... 7497 7498 7499]
  Test:  index=[1500 1501 1502 ... 2997 2998 2999]
  Train err1: 0.673831928462928
  Test err1: 0.6232734806629834
  Train err2: 0.23028074866310158
  Test err2: 0.2082182320441989
Fold 2:
  Train: index=[   0    1    2 ... 7497 7498 7499]
  Test:  index=[3000 3001 3002 ... 4497 4498 4499]
  Train err1: 0.6193314611830782
  Test err1: 0.6548682703321879
  Train err2: 0.2206882664517004
  Test err2: 0.20862929341190214
Fold 3:
  Train: index=[   0    1    2 ... 7497 7498 7499]
  Test:  index=[4500 4501 4502 ... 5997 5998 5999]
  Train err1: 0.6321111069368406
  Test err1: 0.63397781495941
  Train err2: 0.21135036711093638
  Test err2: 0.2829918032786885
Fold 4:
  Train: index=[   0    1    2

### RandomForest

In [68]:
# Without maths features
errs1, errs2 = Eval.evaluate(train, model1=RandomForestClassifier(), model2=RandomForestClassifier())
print(" classfier average accuracy for failure or not: ", np.mean(errs1))
print(" classfier average accuracy for failure type: ", np.mean(errs2))

Fold 0:
  Train: index=[1500 1501 1502 ... 7497 7498 7499]
  Test:  index=[   0    1    2 ... 1497 1498 1499]
  Train err1: 1.0
  Test err1: 0.7535433256095696
  Train err2: 1.0
  Test err2: 0.4151505262299095
Fold 1:
  Train: index=[   0    1    2 ... 7497 7498 7499]
  Test:  index=[1500 1501 1502 ... 2997 2998 2999]
  Train err1: 1.0
  Test err1: 0.7386580960475988
  Train err2: 1.0
  Test err2: 0.46365868631062
Fold 2:
  Train: index=[   0    1    2 ... 7497 7498 7499]
  Test:  index=[3000 3001 3002 ... 4497 4498 4499]
  Train err1: 1.0
  Test err1: 0.7989690721649485
  Train err2: 1.0
  Test err2: 0.4940007255224647
Fold 3:
  Train: index=[   0    1    2 ... 7497 7498 7499]
  Test:  index=[4500 4501 4502 ... 5997 5998 5999]
  Train err1: 0.9975247524752475
  Test err1: 0.7868562929912624
  Train err2: 1.0
  Test err2: 0.49155282331511846
Fold 4:
  Train: index=[   0    1    2 ... 5997 5998 5999]
  Test:  index=[6000 6001 6002 ... 7497 7498 7499]
  Train err1: 1.0
  Test err1: 0.726

In [69]:
# With maths
errs1, errs2 = Eval.evaluate(train, features=Features.make_features, model1=RandomForestClassifier(), model2=RandomForestClassifier())
print(" classfier average accuracy for failure or not: ", np.mean(errs1))
print(" classfier average accuracy for failure type: ", np.mean(errs2))

Fold 0:
  Train: index=[1500 1501 1502 ... 7497 7498 7499]
  Test:  index=[   0    1    2 ... 1497 1498 1499]
  Train err1: 1.0
  Test err1: 0.8135593220338984
  Train err2: 1.0
  Test err2: 0.5665266106442578
Fold 1:
  Train: index=[   0    1    2 ... 7497 7498 7499]
  Test:  index=[1500 1501 1502 ... 2997 2998 2999]
  Train err1: 1.0
  Test err1: 0.8447726306842329
  Train err2: 1.0
  Test err2: 0.6356507059545733
Fold 2:
  Train: index=[   0    1    2 ... 7497 7498 7499]
  Test:  index=[3000 3001 3002 ... 4497 4498 4499]
  Train err1: 1.0
  Test err1: 0.8996563573883162
  Train err2: 0.9814814814814815
  Test err2: 0.5944362601971297
Fold 3:
  Train: index=[   0    1    2 ... 7497 7498 7499]
  Test:  index=[4500 4501 4502 ... 5997 5998 5999]
  Train err1: 1.0
  Test err1: 0.9077275825742084
  Train err2: 1.0
  Test err2: 0.6454918032786886
Fold 4:
  Train: index=[   0    1    2 ... 5997 5998 5999]
  Test:  index=[6000 6001 6002 ... 7497 7498 7499]
  Train err1: 1.0
  Test err1: 0.86

### LightGBM

In [83]:
# With maths
errs1, errs2 = Eval.evaluate(train, features=Features.make_features, model1=LGBMClassifier(), model2=LGBMClassifier())
print(" classfier average accuracy for failure or not: ", np.mean(errs1))
print(" classfier average accuracy for failure type: ", np.mean(errs2))

Fold 0:
  Train: index=[1500 1501 1502 ... 7497 7498 7499]
  Test:  index=[   0    1    2 ... 1497 1498 1499]
[LightGBM] [Info] Number of positive: 189, number of negative: 5811
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002400 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 28567
[LightGBM] [Info] Number of data points in the train set: 6000, number of used features: 129
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.031500 -> initscore=-3.425761
[LightGBM] [Info] Start training from score -3.425761
  Train err1: 1.0
  Test err1: 0.8680745522850786
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002473 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 28567
[LightGBM] [Info] Number of data points in the train set: 6000, number of used features: 129
[LightGBM] [Info] Start training from score -4.621977
[Lig

### Neural network

In [87]:
# With maths
errs1, errs2 = Eval.evaluate(train, features=Features.make_features_naive, model1=MLPClassifier(), model2=MLPClassifier())
print(" classfier average accuracy for failure or not: ", np.mean(errs1))
print(" classfier average accuracy for failure type: ", np.mean(errs2))

Fold 0:
  Train: index=[1500 1501 1502 ... 7497 7498 7499]
  Test:  index=[   0    1    2 ... 1497 1498 1499]
  Train err1: 0.6384907660075445
  Test err1: 0.6140350877192983
  Train err2: 0.36192599550180815
  Test err2: 0.3539732397813495
Fold 1:
  Train: index=[   0    1    2 ... 7497 7498 7499]
  Test:  index=[1500 1501 1502 ... 2997 2998 2999]
  Train err1: 0.5123152709359606
  Test err1: 0.5116279069767442
  Train err2: 0.25092395041795346
  Test err2: 0.23049450549450548
Fold 2:
  Train: index=[   0    1    2 ... 7497 7498 7499]
  Test:  index=[3000 3001 3002 ... 4497 4498 4499]
  Train err1: 0.5182291666666666
  Test err1: 0.5
  Train err2: 0.17483468204175442
  Test err2: 0.16666666666666666
Fold 3:
  Train: index=[   0    1    2 ... 7497 7498 7499]
  Test:  index=[4500 4501 4502 ... 5997 5998 5999]
  Train err1: 0.5329048228526421
  Test err1: 0.52
  Train err2: 0.28179991103480556
  Test err2: 0.2972742200328407
Fold 4:
  Train: index=[   0    1    2 ... 5997 5998 5999]
  Te